In [ ]:
#Google Colab Configure
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repxository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [ ]:
#Google Colab Configure
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [ ]:
#Google Colab Configure
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
#mount gdrive
#unzip from google drive 
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive
!ls my_drive
!unzip ./my_drive/Colab/chs_cut.zip
!ls

In [ ]:
!pip3 install gensim
import numpy as np
import pandas as pd
import gensim
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation,Embedding,SimpleRNN,LSTM,GRU,CuDNNLSTM
from keras.utils import to_categorical
from gensim.models import word2vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
import h5py
import random
from random import shuffle
import os
from glob import glob

In [ ]:
#Get data from processed data.
category={'spam': 0, 'ham': 1}
files = os.listdir("./chs_cut/trec06c")
s=[]

embeddingdim=250

label=[]
t=[]
# 以迴圈處理training data
path='chs_cut/trec06c/*/*/*'
spamData=[]
hamData=[]
for file in glob(path):
    if file.find('spam')>0:
        spamData.append(file)
    elif file.find('ham')>0:
        hamData.append(file)
        
        
shuffle(spamData)
print('Shuffle spamData done.')

print('Put in spam.')
for file in spamData:
    k=''
    with open(file,'r',encoding='utf8') as f:
        k=f.read().replace("\n","/").replace(" ","").split("/")
    while True:
        k_index = k.index('') if '' in k else -1
        if k_index==-1 :break
        k.pop(k_index)
    #print(k)
    
    s+=[k]
    label+=[category['spam']]
print('Put in spam done.')

    
shuffle(hamData)
print('Shuffle hamData done.')



print('Put in ham.')
for file in hamData:
    k=''
    with open(file,'r',encoding='utf8') as f:
        k=f.read().replace("\n","/").replace(" ","").split("/")
    while True:
        k_index = k.index('') if '' in k else -1
        if k_index==-1 :break
        k.pop(k_index)
    #print(k)
    
    s+=[k]
    label+=[category['ham']]
print('Put in ham done.')



print('Start word2vec.')
mod=word2vec.Word2Vec(s,size=embeddingdim, iter=20, sg=1)
mod.save("a.model")
print('Word2vec done.')
#Not know program rate.

In [ ]:
#Move trained model to gdrive.
!ls
!cp ./a.model.trainables.syn1neg.npy ./my_drive/Colab/
!cp ./a.model.wv.vectors.npy ./my_drive/Colab/
!cp ./a.model ./my_drive/Colab/
!ls ./my_drive/Colab/


In [ ]:


#token
templ=len(s)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(s))
sequences = tokenizer.texts_to_sequences(s)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

#Fix the data to a fixed length
data = pad_sequences(sequences,maxlen=200)
data=data[0:templ]

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
label=np.asarray(label)
label = to_categorical(label[indices])

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
#print('Shape of label tensor:', labels.shape)
mod=gensim.models.Word2Vec.load("a.model")

embeddingindex={}

for w,v in mod.wv.vocab.items():
    embeddingindex[w]=mod.wv[w]
    
embeddingmatrix = np.zeros((len(word_index)+1,int(mod.vector_size)))
for w,i in word_index.items():
    
    vec=embeddingindex.get(w)
    if vec is not None:
        embeddingmatrix[i+1] = vec
        
        
embedding_layer = Embedding(len(word_index)+1,
                            embeddingdim,
                            weights=[embeddingmatrix],trainable=False)




model = Sequential()
model.add(embedding_layer)
model.add(LSTM(units=50,unroll=False))
model.add(Dense(units=2, input_dim=50,kernel_initializer='uniform'))
model.add(Activation('sigmoid'))
#model.compile('rmsprop', 'mse')
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

#model.fit(x=data,y=label,batch_size=100, epochs=300, verbose=1, validation_split=0.1)
model.fit(x=data,y=label,batch_size=100, epochs=10, verbose=1,validation_split=0.2)
model.save("m.h5")

print("OK\n")

'''
##predict
y = model.predict_classes(testing)
y = pd.DataFrame(y)
y.to_csv("x.csv",index=True,header=["id,survived"],float_format="%d")

allstr="id,category\n"
k=0
with open("x.csv", "r") as f:
    for i in f.readlines():
        if k==0:
            k=k+1
            continue
        allstr+=i
        
with open("x.csv", "w") as f:
    f.write(allstr)
'''

In [ ]:
#The following used to test.

In [ ]:
!cp ./x.csv ./my_drive/Colab/

In [ ]:
!cp ./m.h5 ./my_drive/Colab/